In [1]:
!pip install PennyLane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.4 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile

In [3]:
import pennylane as qml
from pennylane import numpy as np

In [4]:
import pennylane as qml
from pennylane import numpy as np

In [6]:
n_qubits = 4                # Number of qubits
step = 0.0004               # Learning rate
batch_size = 4              # Number of samples for each training step
num_epochs = 3              # Number of training epochs
q_depth = 6                 # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights

In [7]:
dev = qml.device("default.qubit", wires=n_qubits)

In [8]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)


def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT.
    """
    # In other words it should apply something like :
    # CNOT  CNOT  CNOT  CNOT...  CNOT
    #   CNOT  CNOT  CNOT...  CNOT
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

In [9]:
@qml.qnode(dev, interface="tf")
def quantum_net(q_input_features, q_weights_flat):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    RY_layer(q_input_features)

    # Sequence of trainable variational layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)

In [10]:
class DressedQuantumNet(tf.keras.Model):
    """
    TensorFlow module implementing the *dressed* quantum net.
    """

    def __init__(self):
        """
        Definition of the *dressed* layout.
        """
        super().__init__()
        self.pre_net = tf.keras.layers.Dense(n_qubits, input_shape=(512,))
        self.q_params = tf.Variable(q_delta * tf.random.normal([q_depth * n_qubits]))
        self.post_net = tf.keras.layers.Dense(2)

    def call(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum net.
        """
        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 512 to 4
        pre_out = self.pre_net(input_features)
        q_in = tf.math.tanh(pre_out) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = []
        for elem in tf.unstack(q_in):
            q_out_elem = quantum_net(elem, self.q_params).numpy().astype(np.float32)
            q_out.append(q_out_elem)

        q_out = tf.stack(q_out, axis=0)

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)


In [15]:
model_hybrid = hub.KerasLayer('https://tfhub.dev/google/yamnet/1', trainable=False)  # Load the Yamnet model
model_hybrid.trainable = False  # Freeze all the trainable layers

# Add the DressedQuantumNet as the final layer
model_hybrid = tf.keras.Sequential([
    model_hybrid,
    DressedQuantumNet()
])


In [16]:
criterion = tf.keras.losses.SparseCategoricalCrossentropy()


In [20]:
optimizer_hybrid = tf.keras.optimizers.Adam(learning_rate=step)


In [22]:
model_hybrid.compile(loss=criterion, optimizer=optimizer_hybrid)

In [23]:
model_hybrid.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  multiple                  0         
                                                                 
 dressed_quantum_net_1 (Dres  multiple                 0 (unused)
 sedQuantumNet)                                                  
                                                                 
Total params: 24
Trainable params: 24
Non-trainable params: 0
_________________________________________________________________
